# 5. KPI–OKR Mapping & Implementation
*Aligned with Project Deliverables: KPI–OKR Mapping, Supporting KPIs, and ML-to-Product Metric Mapping*

### 5.1 Strategic Framework: Greener Manufacturing
To ensure our ML project delivers real business value, we map our technical goals directly to the **Mercedes-Benz Ambition 2039** strategy. We are not just predicting numbers; we are optimizing physical factory time to reduce energy usage.

#### **The Objective (O): Optimize Vehicle Testing for Sustainability**
We aim to reduce the time cars spend on test benches, which directly lowers energy consumption (electricity/fuel) and increases factory throughput.

#### **Key Results (KR) & Connected KPIs**
We define success through three distinct layers:

| Key Result (KR) | Supporting KPI (Metric) | Goal | Why it matters |
| :--- | :--- | :--- | :--- |
| **KR 1:** Increase Test Bench Efficiency by 10% | **Mean Absolute Error (MAE)** | Minimize | Shorter tests mean less energy per unit. We need to know (in seconds) how close our prediction is to reality. |
| **KR 2:** Maintain 98% Logistics Reliability | **Under-prediction Rate** | $< 5\%$ | **Critical Risk:** If we predict a test is *short* but it takes *long*, the line stops. This measures the % of dangerous "optimistic" errors. |
| **KR 3:** Validated ML Performance | **$R^2$ Score** | $> 0.55$ | The model must be statistically valid (better than baseline) to be deployed. |

### 5.3 Implementation: The Executive Dashboard
The following code implements this mapping. It transforms raw model predictions into a color-coded dashboard that a Product Manager can read instantly.

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error

def compare_model_kpis(y_true, predictions_dict):
    """
    Generates a comparative KPI dashboard for multiple models (EBM, GBR, Huber).
    
    Args:
    y_true (array-like): Ground truth values.
    predictions_dict (dict): Dictionary of model names and their prediction arrays.
                             e.g., {'EBM': y_pred_ebm, 'GBR': y_pred_gbr...}
    """
    results = []
    
    # Constants for Business Impact
    CO2_FACTOR_PER_SEC = 0.05  # Hypothetical grams of CO2 per second
    RISK_THRESHOLD = 0.05      # Max 5% under-prediction allowed

    for model_name, y_pred in predictions_dict.items():
        # --- 1. Technical Metric (KR3) ---
        r2 = r2_score(y_true, y_pred)
        
        # --- 2. Operational Metric (KR1) ---
        mae = mean_absolute_error(y_true, y_pred)
        
        # --- 3. Risk Metric (KR2 - The "Safety" Check) ---
        # Residual < 0 means we predicted LESS time than it actually took (Blockage Risk)
        residuals = y_pred - y_true
        under_prediction_count = np.sum(residuals < 0)
        under_prediction_rate = under_prediction_count / len(y_true)
        
        # --- 4. Business Impact ---
        total_error_seconds = np.sum(np.abs(residuals))
        wasted_emissions = total_error_seconds * CO2_FACTOR_PER_SEC
        
        # Determine Status based on our hierarchy of needs:
        # 1. MUST PASS Risk Check. 2. MUST PASS R2 Threshold.
        status = "TRACK"
        if under_prediction_rate > RISK_THRESHOLD:
            status = "CRITICAL FAIL" # Risk is too high, regardless of R2
        elif r2 < 0.55:
            status = "FAIL (Low Signal)"
        else:
            status = "PASS"

        results.append({
            "Model": model_name,
            "R2 Score (Tech)": r2,
            "MAE [s] (Efficiency)": mae,
            "Blockage Risk %": under_prediction_rate,
            "Est. Wasted CO2 [g]": wasted_emissions,
            "Status": status
        })

    # Create DataFrame
    df = pd.DataFrame(results)
    
    # --- Styling ---
    # Highlight the "Winner" (Pass status) and "Losers" (Critical Fail)
    def highlight_rows(row):
        if "CRITICAL" in row['Status']:
            return ['background-color: #FFE4E1; color: black'] * len(row) # Light Red
        elif row['Status'] == "PASS":
            return ['background-color: #F0FFF0; color: black'] * len(row) # Honeydew Green
        else:
            return [''] * len(row)

    # Format numbers
    styled_df = df.style.apply(highlight_rows, axis=1)\
        .format({
            "R2 Score (Tech)": "{:.4f}",
            "MAE [s] (Efficiency)": "{:.2f}",
            "Blockage Risk %": "{:.1%}",
            "Est. Wasted CO2 [g]": "{:.2f}"
        })\
        .set_caption("Model Selection Dashboard: Greener Manufacturing")\
        .hide_index()
        
    return styled_df

# --- EXECUTION BLOCK ---
# 1. Prepare your dictionary of predictions
# Replace `y_pred_ebm`, `y_pred_gbr`, `y_pred_huber` with your actual variables
model_predictions = {
    "EBM (Explainable)": y_pred_ebm,    # Replace with your EBM predictions
    "GBR (Boosting)": y_pred_gbr,       # Replace with your GBR predictions
    "Huber (Robust)": y_pred_huber      # Replace with your Huber predictions
}

# 2. Run the comparison
# Ensure `y_val` is your ground truth array
comparison_dashboard = compare_model_kpis(y_val, model_predictions)
display(comparison_dashboard)

NameError: name 'y_pred_ebm' is not defined